In [17]:
import multiprocessing as mp
import numpy as np
import ctypes
import random
import seaborn as sns
import pandas as pd
import time

#### Функция создания общей матрицы, заполненной случайными значениями

In [18]:
def create_matrix ():
    mp_matrix = mp.Array('i', dimension ** 2)
    matrix = np.frombuffer(mp_matrix.get_obj(), ctypes.c_int).reshape(dimension, dimension)
    for i in range(dimension):
        for j in range(dimension):
            matrix[i][j] = random.randint(0, 500)
    return matrix


#### Функции для обычного перемножения двух матриц:

In [19]:
def mul_rows():
    C = np.zeros(dimension**2, ctypes.c_int).reshape(dimension, dimension)
    for i in range(dimension):
        for j in range(dimension):
            for k in range(dimension):
                C[i][j] += A[i][k] * B[k][j]
    return C
def mul_columns():
    C = np.zeros(dimension**2).reshape(dimension, dimension)
    for i in range(dimension):
        for j in range(dimension):
            for k in range(dimension):
                C[j][i] += A[j][k] * B[k][i]
    return C

#### Подсчет времени непараллельного перемножения: по строкам и по столбцам

In [20]:
dimension = 200
A = create_matrix()
B = create_matrix()

time_start = time.time()
C = mul_rows()
time_rows = time.time() - time_start
print("По строкам:", time_rows)

time_start = time.time()
H = mul_columns()
time_columns = time.time() - time_start
print("По столбцам:", time_columns)

По строкам: 5.657637357711792
По столбцам: 8.043640851974487


#### Функция для многопоточного подсчета (каждый процесс считает last_row - first_row + 1 элементов итоговой матрицы)

In [21]:
def mul_process(first_row, last_row):
    for i in range (first_row, last_row + 1):
        for j in range (dimension):
            for k in range(dimension):
                D[i][j] += A[i][k] * B[k][j]

#### Многопоточность достигиается посредством использования структуры multiprocessing.Pool и использования метода starmap_async

In [22]:
times = []
index_list = []
base_times = [time_rows] * 15
for proc_num in range (1, 16):
    index_list.append(proc_num)
    D = mp.Array('i',dimension**2)
    D = np.frombuffer(D.get_obj(), ctypes.c_int).reshape(dimension, dimension)
    part = dimension // proc_num
    remainder = dimension % proc_num
    args = []
    for i in range (0, dimension, part):
        if (i + part - 1) < dimension:
            args.append([i, i + (part-1)])
    args[-1][1]  += remainder
    time_start = time.time()
    pool = mp.Pool(proc_num)
    pool.starmap_async(mul_process, args)
    pool.close()
    pool.join()
    times.append(time.time() - time_start)
    if np.array_equal(C, D):
        print("Подсчет верный")
    else:
        print("Подсчет неверный")
times


Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный
Подсчет верный


[7.625363349914551,
 4.542879819869995,
 3.289067029953003,
 2.6000020503997803,
 2.4585812091827393,
 2.134124517440796,
 2.1301991939544678,
 1.9272851943969727,
 1.8360099792480469,
 1.8792986869812012,
 1.7924470901489258,
 1.9398303031921387,
 1.7918751239776611,
 1.895536184310913,
 1.9732191562652588]

In [31]:
df = pd.DataFrame({
    '': times,
}, index = index_list)

sns.set (rc={'figure.figsize':( 12 , 12 )})
plot = sns.lineplot(data=df)
plot.set(xlabel = "Количество потоков",
        ylabel= "Время вычисления (с.)")
df

AttributeError: module 'seaborn' has no attribute 'set'